In [ ]:
# 变换位置编码后的验证程序
import torch
import numpy as np
# torch.autograd.set_detect_anomaly(True)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import os
import random
import matplotlib.pyplot as plt
import re
from matplotlib.widgets import Slider
from skimage import measure

import plotly.graph_objects as go
import io

class NeRF(nn.Module):
    def __init__(self, D=8, W=256, input_ch=3, input_ch_views=3, output_ch=2, skips=[4], use_viewdirs=True):
        """
        D: 深度，多少层网络
        W: 网络内的channel 宽度
        input_ch: xyz的宽度
        input_ch_views: direction的宽度
        output_ch: 这个参数尽在 use_viewdirs=False的时候会被使用
        skips: 类似resnet的残差连接，表明在第几层进行连接
        use_viewdirs:

        网络输入已经被位置编码后的参数，输入为[64*bs,90]，输出为[64*bs，2]，一位是体积密度，一位是后向散射系数
        """
        super(NeRF, self).__init__()
        self.D = D
        self.W = W
        self.input_ch = input_ch
        self.input_ch_views = input_ch_views
        self.skips = skips
        self.use_viewdirs = use_viewdirs

        # 神经网络,MLP
        # 3D的空间坐标进入的网络
        # 这个跳跃连接层是直接拼接，不是resnet的那种相加
        self.pts_linears = nn.ModuleList(
            [nn.Linear(input_ch, W)] + [nn.Linear(W, W) if i not in self.skips else nn.Linear(W + input_ch, W) for i in
                                        range(D - 1)])

        # 这里channel削减一半 128
        ### Implementation according to the official code release (https://github.com/bmild/nerf/blob/master/run_nerf_helpers.py#L104-L105)
        self.views_linears = nn.ModuleList([nn.Linear(input_ch_views + W, W // 2)])
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

        if use_viewdirs:
            # 特征
            self.feature_linear = nn.Linear(W, W)
            # 体积密度,一个值
            self.alpha_linear = nn.Linear(W, 1)
            # 后向散射系数，一个值
            self.rho_linear = nn.Linear(W // 2, 1)
        else:
            self.output_linear = nn.Linear(W, output_ch)

    def forward(self, x):
        # x [bs*64, 90]
        # input_pts [bs*64, 63]
        # input_views [bs*64,27]
        input_pts, input_views = torch.split(x, [self.input_ch, self.input_ch_views], dim=-1)

        h = input_pts

        for i, l in enumerate(self.pts_linears):

            h = self.pts_linears[i](h)
            h = F.relu(h)
            # 第四层后相加
            if i in self.skips:
                h = torch.cat([input_pts, h], -1)

        if self.use_viewdirs:
            # alpha只与xyz有关
            alpha = self.alpha_linear(h)
            feature = self.feature_linear(h)
            # rho与xyz和d都有关
            h = torch.cat([feature, input_views], -1)

            for i, l in enumerate(self.views_linears):
                h = self.views_linears[i](h)
                h = F.relu(h)

            sigma = self.rho_linear(h)
            alpha = self.relu(alpha) 
            # sigma = 100*self.sigmoid(sigma)
            sigma = self.relu(sigma) 
            
            outputs = torch.cat([alpha, sigma], -1)
        else:
            outputs = self.output_linear(h)

        return outputs


def batchrender(omega,LOS,model,doppler_num):
    '''
    omega为一个[bs,3]变量，指向旋转轴方向，模值为角速度
    LOS为一个[bs,3]变量，方向为视线方向指向物体，模值为1
    model是nerf模型，将一个已经进行位置编码后的位置和视线向量输入进model,可以返回这个位置的体积密度和散射系数
    doppler_num为一个[bs]变量，确定了渲染后光线所在的位置
    '''
    # 确定回波波长
    fc = torch.tensor([9.7e9]).to(device)
    c = torch.tensor([299792458]).to(device)
    lambda0 = c/fc
    # 确定网格参数
    distance_max = 0.6
    distance_min = -0.6
    distance_gap = 100
    doppler_max = 0.15
    doppler_min = -0.15
    doppler_gap = 100
    n_max = 0.60
    n_min = -0.60
    n_gap = 120
    # distance_max = 0.582
    # distance_min = -0.582
    # distance_gap = 97
    # doppler_max = 0.1575
    # doppler_min = -0.1575
    # doppler_gap = 105
    # n_max = 0.60
    # n_min = -0.60
    # n_gap = 120
    # 确定输入batch_size
    batch_size,len = omega.shape
    # 确定每个batch_size输入的投影平面
    omega_norm = torch.linalg.norm(omega,dim = 1)
    omega_normlize = omega/omega_norm.unsqueeze(1)
    Doppler_vector = torch.cross(LOS,omega,dim=1)
    LOSomega_sin_angel = torch.linalg.norm(Doppler_vector,dim=1)/(torch.linalg.norm(omega,dim=1)*torch.linalg.norm(LOS,dim=1))
    Doppler_vector = Doppler_vector/torch.linalg.norm(Doppler_vector,dim = 1).unsqueeze(1)
    # 绘制投影坐标
    distance = torch.linspace(distance_min,distance_max,distance_gap).to(device)
    distance = distance.repeat(batch_size,1)
    distance_delta = torch.tensor((distance_max-distance_min)/distance_gap).to(device)
    doppler = torch.linspace(doppler_min,doppler_max,doppler_gap).repeat(batch_size,1).to(device)
    doppler = doppler*4/LOSomega_sin_angel.unsqueeze(1)
    distance_map = distance.unsqueeze(2)*LOS.unsqueeze(1)
    doppler_map = doppler.unsqueeze(2)*Doppler_vector.unsqueeze(1)
    # 确定投影平面法向量
    n = torch.cross(LOS,Doppler_vector,dim=1)
    n = n/torch.linalg.norm(n,dim = 1).unsqueeze(1)
    # 对投影平面法向量进行随机采样
    n_array = torch.linspace(n_min,n_max,n_gap+1).to(device)
    n_array = n_array.repeat(batch_size,distance_gap,1)
    # 非随机采样
    # n_random_array = n_array[:,:,0:-1] + (n_array[:,:,1:] - n_array[:,:,0:-1])*torch.ones(batch_size,distance_gap,n_gap).to(device)*0.5
    # # 随机采样
    n_random_array = n_array[:,:,0:-1] + (n_array[:,:,1:] - n_array[:,:,0:-1])*torch.rand(batch_size,distance_gap,n_gap).to(device)
    n_random_map = n_random_array.unsqueeze(3)*n.unsqueeze(1).unsqueeze(2)
    # 计算不同随机法向量之间的间隔
    start_n = n.unsqueeze(1).unsqueeze(2)*torch.tensor(n_min).float().to(device)
    start_n = start_n * torch.ones(batch_size,distance_gap,1,3).to(device)
    n_random_map_temp = torch.cat((start_n,n_random_map),dim=2)
    n_delta = torch.norm(n_random_map_temp[:,:,0:-1,:]-n_random_map,dim=3)

    # 计算所有需要输入网络的坐标
    code_flag = 1
    if code_flag == 1:
        xyz = doppler_map[torch.arange(batch_size),doppler_num,:].unsqueeze(1).unsqueeze(2) + distance_map.unsqueeze(2) + n_random_map
        xyz_coding = positon_code_xyz(xyz)
        LOS_coding = position_code_LOS(LOS)
        LOS_coding = ((LOS_coding.unsqueeze(1).unsqueeze(2))*torch.ones(batch_size,distance_gap,n_gap,27).to(device)).view(-1,27)
        xyzLOS_coding = torch.cat((xyz_coding,LOS_coding),dim=1)
    else:
        xyz = doppler_map[torch.arange(batch_size),doppler_num,:].unsqueeze(1).unsqueeze(2) + distance_map.unsqueeze(2) + n_random_map
        xyz_coding = xyz.view(-1,3)
        LOS_coding = ((LOS.unsqueeze(1).unsqueeze(2))*torch.ones(batch_size,distance_gap,n_gap,3).to(device)).view(-1,3)
        print(LOS_coding)
        xyzLOS_coding = torch.cat((xyz_coding,LOS_coding),dim=1)
    output = model(xyzLOS_coding)
    output = output.view(batch_size,distance_gap,n_gap,2)
    render_equaltion = 1
    if render_equaltion == 0:
        Ti = torch.cumprod(torch.exp(-output[:,:,:,0]*distance_delta),dim=1)
        distance_profile = torch.sum(output[:,:,:,0]*(1-torch.exp(-output[:,:,:,1]*n_delta))*Ti,dim=2)
    elif render_equaltion == 1:
        Ti = torch.cumprod(torch.exp(-output[:,:,:,0]**2*distance_delta),dim=1)
        temp = output[:,:,:,0]*output[:,:,:,1]*n_delta*Ti
        distance_profile = torch.sum(output[:,:,:,0]*output[:,:,:,1]*n_delta*Ti,dim=2)
    elif render_equaltion == 2:
        Ti = torch.cumprod(torch.exp(-output[:,:,:,0]*distance_delta),dim=1)
        # print(Ti[:,0,:])
        # 将Ti的第1维首增加一个1，并去除最后一维，方便计算
        Ti = torch.cat((torch.ones(batch_size,1,n_gap).to(device),Ti),dim=1)[:,:-1,:]
        # 计算alpha_i
        alphai = 1-torch.exp(-output[:,:,:,0]*distance_delta)
        temp = alphai*output[:,:,:,1]*n_delta*Ti
        distance_profile = torch.sum(temp,dim=2)
    elif render_equaltion == 3:
        # Ranerf的累积规则
        Ti = torch.cumprod(torch.exp(-output[:,:,:,0]*distance_delta),dim=2)
        Ti = torch.cat((torch.ones(batch_size,distance_gap,1).to(device),Ti),dim=2)[:,:,:-1]
        alphai = 1-torch.exp(-output[:,:,:,0]*n_delta)
        temp = alphai*output[:,:,:,1]*n_delta*Ti
        distance_profile = torch.sum(alphai*output[:,:,:,1]*n_delta*Ti,dim=2)    
    return distance_profile,temp[:,:,:]
    # return distance_profile,output[:,:,:,0]


def positon_code_xyz(xyz):
    code_len = 10
    batch_size,distance,n,dimension = xyz.shape
    xyz = xyz.view(-1,dimension)
    xyz = xyz
    position_coding = torch.zeros_like(xyz).to(device)
    position_coding = position_coding.repeat(1,code_len*2)
    div_term = 2 ** torch.arange(0,code_len,step=1).to(device)
    position_coding[:,0::2] = torch.sin((xyz.unsqueeze(1) * math.pi * div_term.unsqueeze(1).unsqueeze(0)).view(batch_size*distance*n,-1))
    position_coding[:,1::2] = torch.cos((xyz.unsqueeze(1) * math.pi * div_term.unsqueeze(1).unsqueeze(0)).view(batch_size*distance*n,-1))
    position_coding = torch.cat((xyz,position_coding),dim=1)
    
    return position_coding

def position_code_LOS(LOS):
    code_len = 4
    batch_size,dimension = LOS.shape
    position_coding = torch.zeros_like(LOS).to(device)
    position_coding = position_coding.repeat(1,code_len*2)
    div_term = 2 ** torch.arange(0,code_len,step=1).to(device)
    position_coding[:,0::2] = torch.sin((LOS.unsqueeze(1) * div_term.unsqueeze(1).unsqueeze(0)).view(batch_size,-1))
    position_coding[:,1::2] = torch.cos((LOS.unsqueeze(1) * div_term.unsqueeze(1).unsqueeze(0)).view(batch_size,-1))
    position_coding = torch.cat((LOS,position_coding),dim=1)

    return position_coding

def picture_sample(images,LOS_dirs,omegas,batch_size,image_hight = 100,image_width = 100, image_num = 30,temp_num = None):
    # temp_num = random.sample(range(image_num),1)
    # temp_num = [54]
    print(temp_num)
    temp = [t*image_hight for t in temp_num] + np.arange(100)
    data_num = [x//image_hight for x in temp]
    doppler_numbers = [x % image_hight for x in temp]
    
    LOS_dirs_batch = [LOS_dirs[x] for x in data_num]
    omegas_batch = [omegas[x] for x in data_num]
    range_profile_batch = [images[x][y,:] for x,y in zip(data_num,doppler_numbers)]


    omegas_batch_tensor = torch.stack(omegas_batch).to(device)
    LOS_dirs_batch_tensor = torch.stack(LOS_dirs_batch).to(device)
    range_profile_batch_tensor = torch.stack(range_profile_batch).to(device)
    doppler_profil_num_tensor = torch.tensor(doppler_numbers).long().to(device)

    # range_image = range_profile_batch_tensor.detach().cpu()
    # plt.imshow(range_image)
    # plt.show()
    
    return omegas_batch_tensor,LOS_dirs_batch_tensor,range_profile_batch_tensor,doppler_profil_num_tensor

def natural_sort_key(s):
    # 分割字符串中的数字并将它们转换为整数
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def loaddata(folder_path):
    '''
    输入文件夹路径，输出数据集
    '''
    # 获取文件夹中的所有文件和子文件夹
    items = os.listdir(folder_path)
    # 过滤出所有文件（排除子文件夹）
    files = [item for item in items if os.path.isfile(os.path.join(folder_path, item)) and item.endswith('.npz')]
    files_sorted = sorted(files, key=natural_sort_key)
    # files_sorted = random.sample(files_sorted, 16)
    #载入数据
    images = []
    LOS_dirs = []
    omegas = []
    for file in files_sorted:
        full_path = folder_path+"/"+file
        data = np.load(full_path)
        image = torch.from_numpy(data['image']).to(device)
        LOS_dir = torch.from_numpy(data['LOS']).to(device)
        omega = torch.from_numpy(data['rotation_axis']).to(device)
        images.append(image)
        LOS_dirs.append(LOS_dir)
        omegas.append(omega)

    # 可视化LOS_dirs
    LOS_dirs = torch.stack(LOS_dirs)
    print(LOS_dirs.shape)
    # 在三维空间中表示LOS_dirs
    fig = go.Figure(data=[go.Scatter3d(x=LOS_dirs[:, 0].cpu().numpy(),
                                        y=LOS_dirs[:, 1].cpu().numpy(),
                                        z=LOS_dirs[:, 2].cpu().numpy(),
                                        mode='markers',
                                        marker=dict(size=2, color='blue'))])
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
                      title='LOS Directions in 3D Space')
    fig.show()
    



    return images,LOS_dirs,omegas

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

modelname = 'experiment210'

model = NeRF(input_ch = 63, input_ch_views = 27, use_viewdirs = True).to(device)
model.load_state_dict(torch.load('/DATA/disk1/asteroid/asteroid_inverse/Instant-ngp/model/'+ modelname +'/model_state_dict.pth'))
model.eval()

omega_real = math.pi/900

folder_path = '/DATA/disk1/asteroid/asteroid_inverse/ImageGen/3dmodel/XXX/XXX_dilate_real_image_13.8du'
# folder_path = '/DATA/disk1/asteroid/asteroid_inverse/Instant-ngp/new_dataset/sys_data/contactball_rot90'

images,LOS_dirs,omegas = loaddata(folder_path)

# 获得list类型数据omegas的长度
image_num = len(images)

# omegas_batch_tensor,LOS_dirs_batch_tensor,range_profile_batch_tensor,doppler_profil_num_tensor = picture_sample(images,LOS_dirs,omegas,batch_size = 40,image_num=image_num)
# distance_profile_batch,output = batchrender(omegas_batch_tensor*omega_real,LOS_dirs_batch_tensor,model,doppler_profil_num_tensor)

# # print(LOS_dirs)
# # print(LOS_dirs_batch_tensor)

# range_image1 = distance_profile_batch.detach().cpu()
# # plt.figure()
# # plt.imshow(range_image1)
# # plt.colorbar()


# range_image2 = range_profile_batch_tensor.detach().cpu()



# ray_distribution = output.detach().cpu().numpy()
# # ray_distribution_one = ray_distribution[40,78,:]

# # # 绘制图像
# # plt.figure()
# # plt.plot(ray_distribution_one)
# # plt.show()




In [ ]:
# 根据LOS_dir计算需要旋转的角度
def calculate_rotation_angle(LOS_dir):
    """
    计算LOS方向向量与X轴之间的旋转角度（顺时针）
    
    参数:
        LOS_dir: 3D LOS方向向量 (numpy array)
    
    返回:
        旋转角度（度）
    """
    # 计算LOS方向向量在XZ平面上的投影
    xz_projection = np.array([LOS_dir[0],0, LOS_dir[2]])
    
    # 计算投影与原向量之间的夹角
    if np.linalg.norm(xz_projection) == 0:
        return 0, 0  # 如果投影为零向量，返回0角度
    xz_projection = xz_projection / np.linalg.norm(xz_projection)  # 归一化投影向量
    LOS_dir = LOS_dir / np.linalg.norm(LOS_dir)  # 归一化LOS方向向量
    # 计算投影与LOS方向向量的夹角
    angle_rad1 = np.arccos(np.clip(np.dot(xz_projection, LOS_dir), -1.0, 1.0))
    
    angle_deg1 = np.degrees(angle_rad1)  # 转换为度数

    # 计算投影的余弦值
    angle_rad2 = np.arctan2(xz_projection[2], xz_projection[0])
    angle_deg2 = np.degrees(angle_rad2)
    
    return angle_deg1, angle_deg2

def merge_density_volumes_and_extract_mesh(images, LOS_dirs, omegas, model, 
                                          num_views=8, grid_size=120, threshold_ratio=0.3):
    """
    通过体素融合多个视角的密度信息，然后提取统一网格
    
    参数:
        images, LOS_dirs, omegas: 输入数据
        model: NeRF模型
        num_views: 融合的视角数量
        grid_size: 体素网格大小
        threshold_ratio: 最终提取网格的阈值
    """
    # 创建3D体素网格
    fused_density = np.zeros((grid_size, grid_size, grid_size))
    weight_grid = np.zeros((grid_size, grid_size, grid_size))
    
    # 选择多个视角
    selected_indices = random.sample(range(len(images)), min(num_views, len(images)))
    
    for i, idx in enumerate(selected_indices):
        print(f"融合第 {i+1}/{num_views} 个视角...")

        omegas_batch_tensor,LOS_dirs_batch_tensor,range_profile_batch_tensor,doppler_profil_num_tensor = picture_sample(images,LOS_dirs,omegas,batch_size = 40,image_num=image_num,temp_num=[idx])
        
        # # 生成当前视角的密度分布
        # omegas_batch = [omegas[idx]]
        # LOS_dirs_batch = [LOS_dirs[idx]]
        # doppler_numbers = [50]
        
        # omegas_batch_tensor = torch.stack(omegas_batch).to(device)
        # LOS_dirs_batch_tensor = torch.stack(LOS_dirs_batch).to(device)
        # doppler_profil_num_tensor = torch.tensor(doppler_numbers).long().to(device)
        
        # 渲染
        distance_profile_batch, output = batchrender(
            omegas_batch_tensor * omega_real,
            LOS_dirs_batch_tensor,
            model,
            doppler_profil_num_tensor
        )
        
        ray_distribution = output[0].detach().cpu().numpy()
        
        # 计算权重（基于视角质量或其他因素）
        weight = 1.0  # 可以根据需要调整权重
        
        # 将当前密度添加到融合网格
        # 这里需要将ray_distribution变换到统一的世界坐标系
        LOS_dir = LOS_dirs_batch_tensor[0,:].cpu().numpy()
        angle1, angle2 = calculate_rotation_angle(LOS_dir)
        
        # 变换密度网格到世界坐标
        transformed_density = transform_density_to_world(ray_distribution, angle1, angle2, grid_size)
        
        # 融合
        fused_density += transformed_density * weight
        weight_grid += weight
    
    # 归一化
    valid_mask = weight_grid > 0
    fused_density[valid_mask] /= weight_grid[valid_mask]
    
    # 从融合密度提取网格
    vmin, vmax = fused_density.min(), fused_density.max()
    threshold = vmin + (vmax - vmin) * threshold_ratio
    
    verts, faces, normals, values = measure.marching_cubes(fused_density, threshold)
    verts = verts / grid_size * 1.2 - 0.6
    
    return verts, faces, fused_density

def transform_density_to_world(ray_distribution, angle1, angle2, target_size):
    """
    将局部密度分布变换到世界坐标系
    """
    # 这是一个简化实现，实际需要根据具体的坐标变换来实现
    # 可能需要使用插值来重新采样到统一网格
    from scipy.ndimage import zoom
    
    # 简单的尺寸调整
    if ray_distribution.shape != (target_size, target_size, target_size):
        scale_factors = [target_size / s for s in ray_distribution.shape]
        transformed = zoom(ray_distribution, scale_factors)
    else:
        transformed = ray_distribution.copy()
    
    return transformed


print("开始体素融合...")
fused_vertices, fused_faces, fused_density = merge_density_volumes_and_extract_mesh(
    images, LOS_dirs, omegas, model,
    num_views=8, grid_size=120, threshold_ratio=0.3
)

# 保存融合模型
if fused_vertices is not None:
    fused_output_path = f'/DATA/disk1/asteroid/asteroid_inverse/Instant-ngp/model/{modelname}/fused_model.obj'
    with open(fused_output_path, 'w') as f:
        f.write(f"# 体素融合模型，顶点数: {len(fused_vertices)}, 面数: {len(fused_faces)}\n\n")
        
        for v in fused_vertices:
            f.write(f"v {v[0]} {v[1]} {v[2]}\n")
        
        for face in fused_faces:
            f.write(f"f {face[0]+1} {face[1]+1} {face[2]+1}\n")
    
    print(f"融合模型已保存: {fused_output_path}")